# ICL Step

In [1]:
import json
from functions import *
with open('math_train_1.json') as f:
    gold_dat = json.load(f)
with open('train_math_llama2_7b_full_weak.json') as f:
    llama_dat = json.load(f)
with open('train_math_gemma_2b_full_weak.json') as f:
    gemma_dat = json.load(f)
    #print(d)
with open('train_math_mistral_7b_full_weak.json') as f:
    mistral_dat = json.load(f)

In [ ]:
gold_quest = []
gold_answers = []
for i in range(500):
    dct = gold_dat[i]
    gold_quest.append(dct['content'])
    gold_answers.append(dct['output'])

In [ ]:
system_prompt = ''''''
SaveJSONL(system_prompt, gold_quest, gold_answers, 'MATH_gold.jsonl')

In [ ]:
#getting gpt-3.5-turbo data
training_questions = [llama_dat[i]['content'] for i in range(0, 203)]
gpt_ans = []
for question in training_questions:
    system_prompt = ''''''
    gpt_prompt = FormatInput(system_prompt, question, gpt_35_teacher_id)
    gpt_ans.append(QueryModel(gpt_prompt, gpt_35_teacher_id, api='OPENAI'))
SaveJSONL(system_prompt, training_questions, gpt_ans, 'gpt_3.5_weak.jsonl')

In [11]:
import json
with open('gpt_3.5_weak.jsonl') as f:
    gpt_data = [json.loads(line) for line in f]

In [24]:
llama_icl_examples, gemma_icl_examples, mistral_icl_examples, gpt_ICL_examples = [], [], [], []
for i in [0,2]:
    llama_icl_examples.append(llama_dat[i]['content']+llama_dat[i]['output'])
    gemma_icl_examples.append(gemma_dat[i]['content']+gemma_dat[i]['output'])
    mistral_icl_examples.append(mistral_dat[i]['content']+mistral_dat[i]['output'])
    gpt_ICL_examples.append(gpt_data[0]['messages'][1]['content']+gpt_data[0]['messages'][2]['content'])

In [25]:
# getting the training examples, I use the first 200 samples exluding the icl examples 
training_questions = [llama_dat[i]['content'] for i in range(3, 203)]
llama_ans = [llama_dat[i]['output'] for i in range(3, 203)]
gemma_ans = [gemma_dat[i]['output'] for i in range(3, 203)]
mistral_ans = [mistral_dat[i]['output'] for i in range(3, 203)]

In [5]:
#getting ICL data
improvement_model = 'gpt-4o-mini-2024-07-18'
ICL_llama_ans = []
ICL_gemma_ans = []
ICL_mistral_ans = []
for question in training_questions:
    system_prompt = ''''''
    llama_ICL_prompt = '\n\n'.join(llama_icl_examples)
    gemma_ICL_prompt = '\n\n'.join(gemma_icl_examples)
    mistral_ICL_prompt = '\n\n'.join(mistral_icl_examples)
    llama_prompt = llama_ICL_prompt+question
    gemma_prompt = gemma_ICL_prompt+question
    mistral_prompt = mistral_ICL_prompt+question
    llama_prompt = FormatInput(system_prompt, llama_prompt, improvement_model)
    gemma_prompt = FormatInput(system_prompt, gemma_prompt, improvement_model)
    mistral_prompt = FormatInput(system_prompt, mistral_prompt, improvement_model)
    ICL_llama_ans.append(QueryModel(llama_prompt, improvement_model, api='OPENAI'))
    ICL_gemma_ans.append(QueryModel(gemma_prompt, improvement_model, api='OPENAI'))
    ICL_mistral_ans.append(QueryModel(mistral_prompt, improvement_model, api='OPENAI'))
SaveJSONL(system_prompt, training_questions, ICL_llama_ans, 'math_llama2_mini_ICL.jsonl')
SaveJSONL(system_prompt, training_questions, llama_ans, 'math_llama2_mini_weak.jsonl')
SaveJSONL(system_prompt, training_questions, ICL_gemma_ans, 'math_gemma_mini_ICL.jsonl')
SaveJSONL(system_prompt, training_questions, gemma_ans, 'math_gemma_mini_weak.jsonl')
SaveJSONL(system_prompt, training_questions, ICL_mistral_ans, 'math_mistral_mini_ICL.jsonl')
SaveJSONL(system_prompt, training_questions, mistral_ans, 'math_mistral_mini_weak.jsonl')

In [28]:
# getting gpt ICL data
improvement_model = 'gpt-4o-mini-2024-07-18'
ICL_gpt_ans = []
for question in training_questions:
    system_prompt = ''''''
    gpt_ICL_prompt = '\n\n'.join(gpt_ICL_examples)
    gpt_prompt = gpt_ICL_prompt+question
    gpt_prompt = FormatInput(system_prompt, gpt_prompt, improvement_model)
    ICL_gpt_ans.append(QueryModel(gpt_prompt, improvement_model, api='OPENAI'))
SaveJSONL(system_prompt, training_questions, ICL_gpt_ans, 'math_gpt_mini_ICL.jsonl')

# FineTuning

In [6]:
improvement_model = 'gpt-4o-mini-2024-07-18'
frac_train = 1 #Do not modify this. Validation data will not be used anyways...

In [30]:
CheckTokens(f'math_llama2_mini_weak.jsonl')
CheckTokens(f'math_llama2_mini_ICL.jsonl')
CheckTokens(f'math_gemma_mini_weak.jsonl')
CheckTokens(f'math_gemma_mini_ICL.jsonl')
CheckTokens(f'math_mistral_mini_weak.jsonl')
CheckTokens(f'math_mistral_mini_ICL.jsonl')
CheckTokens(f'math_gpt_mini_ICL.jsonl')
#CheckTokens(f'MATH_gold.jsonl')

Processing file: math_llama2_mini_weak.jsonl

#### Distribution of total tokens:
min / max: 70, 2379
mean / median: 601.35, 238.0
p5 / p95: 112.80000000000001, 2050.6

#### Distribution of assistant tokens:
min / max: 32, 2282
mean / median: 512.3, 132.0
p5 / p95: 55.0, 1963.0
**************************************************
Processing file: math_llama2_mini_ICL.jsonl

#### Distribution of total tokens:
min / max: 133, 2117
mean / median: 655.78, 565.5
p5 / p95: 267.0, 1148.2

#### Distribution of assistant tokens:
min / max: 100, 1999
mean / median: 566.73, 471.5
p5 / p95: 201.9, 1014.8
**************************************************
Processing file: math_gemma_mini_weak.jsonl

#### Distribution of total tokens:
min / max: 59, 2553
mean / median: 512.315, 207.0
p5 / p95: 103.0, 1953.4

#### Distribution of assistant tokens:
min / max: 15, 2434
mean / median: 423.265, 125.0
p5 / p95: 55.900000000000006, 1847.6
**************************************************
Processing file: mat

In [8]:
#math_llama_weak_fine_tune_job = FineTune('math_llama2_mini_weak.jsonl')
math_llama_mini_weak_id = 

In [9]:
#math_llama_ICL_fine_tune_job = FineTune('math_llama2_mini_ICL.jsonl')
math_llama_mini_ICL_id = 

In [10]:
#math_gemma_weak_fine_tune_job = FineTune('math_gemma_mini_weak.jsonl')
math_gemma_mini_weak_id = 

In [11]:
#math_gemma_ICL_fine_tune_job = FineTune('math_gemma_mini_ICL.jsonl')
math_gemma_mini_ICL_id = 

In [12]:
#math_mistral_weak_fine_tune_job = FineTune('math_mistral_mini_weak.jsonl')
math_mistral_mini_weak_id = 

In [13]:
#math_mistral_ICL_fine_tune_job = FineTune('math_mistral_mini_ICL.jsonl')
math_mistral_mini_ICL_id = 

In [32]:
#math_gpt_ICL_fine_tune_job = FineTune('math_gpt_mini_ICL.jsonl')
math_gpt_mini_ICL_id = 

# Test Responses

In [33]:
with open('test.jsonl') as f:
    test_data = [json.loads(line) for line in f]
#test_data[0]
small_test = [test_data[i]['problem'] for i in range(100)]
small_test_key = [test_data[i]['solution'] for i in range(100)]

In [35]:
##Construct test data
llama_ICL_gpt_test_ans = []
llama_weak_gpt_test_ans = []
gemma_weak_gpt_test_ans = []
gemma_ICL_gpt_test_ans = []
mistral_weak_gpt_test_ans = []
mistral_ICL_gpt_test_ans = []
baseline_ans = []
for i, question in enumerate(small_test):
    system_prompt = ''''''
    #print(question)
    user_prompt = question
    key = small_test_key[i]
    prompt = FormatInput(system_prompt, user_prompt,'gpt-4o-mini')
    llama_ICL_gpt_test_ans.append(QueryModel(prompt, math_llama_mini_ICL_id, api='OPENAI'))
    llama_weak_gpt_test_ans.append(QueryModel(prompt, math_llama_mini_weak_id, api='OPENAI'))
    gemma_ICL_gpt_test_ans.append(QueryModel(prompt, math_gemma_mini_ICL_id,api = 'OPENAI'))
    gemma_weak_gpt_test_ans.append(QueryModel(prompt, math_gemma_mini_weak_id,api = 'OPENAI'))
    mistral_weak_gpt_test_ans.append(QueryModel(prompt, math_mistral_mini_weak_id,api = 'OPENAI'))
    mistral_ICL_gpt_test_ans.append(QueryModel(prompt, math_mistral_mini_ICL_id,api = 'OPENAI'))
    baseline_ans.append(QueryModel(prompt, 'gpt-4o-mini-2024-07-18',api='OPENAI'))
SaveJSONL(system_prompt, small_test, llama_ICL_gpt_test_ans, 'math_llama2_mini_ICL_test.jsonl')
SaveJSONL(system_prompt, small_test, llama_weak_gpt_test_ans, 'math_llama2_mini_weak_test.jsonl')
SaveJSONL(system_prompt, small_test, baseline_ans, 'math_mini_baseline_test.jsonl')
SaveJSONL(system_prompt, small_test, mistral_weak_gpt_test_ans, 'math_mistral_mini_weak_test.jsonl')
SaveJSONL(system_prompt, small_test, mistral_ICL_gpt_test_ans, 'math_mistral_mini_ICL_test.jsonl')
SaveJSONL(system_prompt, small_test, gemma_weak_gpt_test_ans, 'math_gemma_mini_weak_test.jsonl')
SaveJSONL(system_prompt, small_test, gemma_ICL_gpt_test_ans, 'math_gemma_mini_ICL_test.jsonl')

In [34]:
gpt_ICL_mini_test_ans = []
for i, question in enumerate(small_test):
    system_prompt = ''''''
    #print(question)
    user_prompt = question
    key = small_test_key[i]
    prompt = FormatInput(system_prompt, user_prompt,'gpt-4o-mini')
    gpt_ICL_mini_test_ans.append(QueryModel(prompt, math_gpt_mini_ICL_id, api='OPENAI'))

NameError: name 'gemma_ICL_gpt_test_ans' is not defined

In [35]:
SaveJSONL(system_prompt, small_test, gpt_ICL_mini_test_ans, 'math_gpt_mini_ICL_test.jsonl')

# Eval

In [37]:
with open('math_llama2_mini_ICL_test.jsonl') as f:
    llama_ICL_test_resp = [json.loads(line) for line in f]
with open('math_llama2_mini_weak_test.jsonl') as f:
    llama_weak_test_resp = [json.loads(line) for line in f]
with open('math_mini_baseline_test.jsonl') as f:
    baseline_resp = [json.loads(line) for line in f]
with open('math_mistral_mini_weak_test.jsonl') as f:
    mistral_weak_test_resp = [json.loads(line) for line in f]
with open('math_mistral_mini_ICL_test.jsonl') as f:
    mistral_ICL_test_resp = [json.loads(line) for line in f]
with open('math_gemma_mini_weak_test.jsonl') as f:
    gemma_weak_test_resp = [json.loads(line) for line in f]
with open('math_gemma_mini_ICL_test.jsonl') as f:
    gemma_ICL_test_resp = [json.loads(line) for line in f]
with open('math_gpt_mini_ICL_test.jsonl') as f:
    gpt_ICL_test_resp = [json.loads(line) for line in f]

In [40]:
llama_ICL_scores, llama_weak_scores, baseline = [], [], []
mistral_ICL_scores, mistral_weak_scores, gemma_ICL_scores, gemma_weak_scores = [], [], [], []
eval_model = 'gpt-4o'
for i, testq in enumerate(small_test):
    eval_sys = GetEvalSystemPrompt()
    key = small_test_key[i]
    eval_prompt_baseline = FormatInput(eval_sys, GetEvalUserPrompt(testq, key, baseline_resp[i]['messages'][2]['content']), model = eval_model)
    eval_prompt_llama_weak = FormatInput(eval_sys, GetEvalUserPrompt(testq, key, llama_weak_test_resp[i]['messages'][2]['content']), model = eval_model)
    eval_prompt_llama_ICL = FormatInput(eval_sys, GetEvalUserPrompt(testq, key, llama_ICL_test_resp[i]['messages'][2]['content']), model=eval_model)
    try: #sometimes the gemma trained model fails to provide an answer
        eval_prompt_gemma_weak = FormatInput(eval_sys, GetEvalUserPrompt(testq, key, gemma_weak_test_resp[i]['messages'][2]['content']), model = eval_model)
        gemma_weak_scores.append(GPTEval(eval_prompt_gemma_weak, model=eval_model))
    except: # if answer is blank score 0
        print(i)
        gemma_weak_scores.append(0)
    eval_prompt_gemma_ICL = FormatInput(eval_sys, GetEvalUserPrompt(testq, key, gemma_ICL_test_resp[i]['messages'][2]['content']), model = eval_model)
    eval_prompt_mistral_weak = FormatInput(eval_sys, GetEvalUserPrompt(testq, key, mistral_weak_test_resp[i]['messages'][2]['content']), model = eval_model)
    eval_prompt_mistral_ICL = FormatInput(eval_sys, GetEvalUserPrompt(testq, key, mistral_ICL_test_resp[i]['messages'][2]['content']), model = eval_model)
    llama_ICL_scores.append(GPTEval(eval_prompt_llama_ICL, model=eval_model))
    llama_weak_scores.append(GPTEval(eval_prompt_llama_weak, model=eval_model))
    baseline.append(GPTEval(eval_prompt_baseline, model=eval_model))
    mistral_ICL_scores.append(GPTEval(eval_prompt_mistral_ICL, model=eval_model))
    mistral_weak_scores.append(GPTEval(eval_prompt_mistral_weak, model=eval_model))
    gemma_ICL_scores.append(GPTEval(eval_prompt_gemma_ICL, model=eval_model))
    #gemma_weak_scores.append(GPTEval(eval_prompt_gemma_weak, model=eval_model))

In [38]:
eval_model = 'gpt-4o'
scores = []
for i, testq in enumerate(small_test):
    eval_sys = GetEvalSystemPrompt()
    key = small_test_key[i]
    eval_prompt_gpt = FormatInput(eval_sys, GetEvalUserPrompt(testq, key, gpt_ICL_test_resp[i]['messages'][2]['content']), model = eval_model)
    scores.append(GPTEval(eval_prompt_gpt, model=eval_model))

In [41]:
arrays = [llama_ICL_scores, llama_weak_scores, baseline, mistral_ICL_scores, mistral_weak_scores, gemma_ICL_scores, gemma_weak_scores]

In [42]:
print('baseline:'+str(sum(baseline)/100)+' +/- '+str(2*np.std(baseline)/10))
print('lweak:'+str(sum(llama_weak_scores)/100)+' +/- '+str(2*np.std(llama_weak_scores)/10))
print('lICL:'+str(sum(llama_ICL_scores)/100)+' +/- '+str(2*np.std(llama_ICL_scores)/10))
print('gweak:'+str(sum(gemma_weak_scores)/100)+' +/- '+str(2*np.std(gemma_weak_scores)/10))
print('gICL:'+str(sum(gemma_ICL_scores)/100)+' +/- '+str(2*np.std(gemma_ICL_scores)/10))
print('mweak:'+str(sum(mistral_weak_scores)/100)+' +/- '+str(2*np.std(mistral_weak_scores)/10))
print('mICL:'+str(sum(mistral_ICL_scores)/100)+' +/- '+str(2*np.std(mistral_ICL_scores)/10))

baseline:[0.78344444] +/- 0.07967046016072847
lweak:[0.06789474] +/- 0.04964795733552417
lICL:[0.7415] +/- 0.08618648385912955
gweak:[0.30013158] +/- 0.0893730509928318
gICL:[0.7395] +/- 0.08575890624302526
mweak:[0.35194737] +/- 0.09159445180308703
mICL:[0.7295] +/- 0.08616373947316817


In [69]:
dictionary = {'baseline':baseline, 'lweak': llama_weak_scores, 'lICL': llama_ICL_scores, 'gweak': gemma_weak_scores, 'gICL:': gemma_ICL_scores, 'mweak': mistral_weak_scores, 'mICL': mistral_ICL_scores}
np.save('math_scores_mini.npy', dictionary) 

In [39]:
print(sum(scores)/100)

[0.74002632]
